# word2vec, fastTextを用いた日本語単語のベクトル表現の実装

## 目標
1.	学習済みの日本語word2vecモデルで単語をベクトル表現に変換する実装ができるようになる
2.	学習済みの日本語fastText モデルで単語をベクトル表現に変換する実装ができるようになる

## Library

In [3]:
import MeCab
import re
import torchtext

In [15]:
# 単語分割にMeCab+NEologdを使用
m_t = MeCab.Tagger('-Owakati -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')

# 単語分割
def tokenizer_mecab(text):
    text = m_t.parse(text)  # スペースが単語で区切られる
    ret = text.strip().split()  # スペース部分で区切ったリストに変換
    
    return ret


# 前処理として正規化する関数
def preprocessing_text(text):
    # 改行、半角スペース、全角スペースを削除
    text = re.sub('\r', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('　', '', text)
    text = re.sub(' ', '', text)

    # 数字文字の一律「0」化
    text = re.sub(r'[0-9 ０-９]', '0', text)  # 数字

    return text


#  前処理（正規化）と単語分割を合わせた関数を定義する(上の２つの関数)
def tokenizer_with_preprocessing(text):
    text = preprocessing_text(text)  # 正規化
    ret = tokenizer_mecab(text)       # 単語分割 
        
    return ret


In [16]:
# tsv, csvデータを読み込んだときに読み込んだ内容に対して行う処理を定義する
max_length = 25
TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing, 
                           use_vocab=True, lower=True, include_lengths=True, batch_first=True, fix_length=max_length)
LABEL = torchtext.data.Field(sequential=False, use_vocab=False)

# 1行がTEXTとLABELで区切られていることをfieldsで指示する
# FieldとそのFieldに対する処理（上で定義したもの）
train_ds, val_ds, test_ds = torchtext.data.TabularDataset.splits(
    path='./data/', train='text_train.tsv', 
    validation='text_val.tsv', test='text_test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])

## 単語のベクトル化

### word2vec
torchtextの学習済みモデルは精度が微妙なので東北大学 乾・岡崎研究室で公開されているWord2Vecの学習済みのベクトルを使用

In [17]:
# そのままではtorchtextで読み込めないので、gensimライブラリを使用して、
# Word2Vecのformatで保存し直します

# 事前インストール
# pip install gensim

from gensim.models import KeyedVectors


# 一度gensimライブラリで読み込んで、word2vecのformatで保存する
model = KeyedVectors.load_word2vec_format(
    './data/entity_vector/entity_vector.model.bin', binary=True)

# 保存（時間がかかります、10分弱）　インスタンス上でやると速かった　１分ぐらい
model.wv.save_word2vec_format('./data/japanese_word2vec_vectors.vec')

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  from ipykernel import kernelapp as app


In [18]:
# torchtextで単語ベクトルとして読み込む
from torchtext.vocab import Vectors

japanese_word2vec_vectors = Vectors(
    name='./data/japanese_word2vec_vectors.vec')

# 単語ベクトルの中身を確認
print('１単語を表現する次元数：', japanese_word2vec_vectors.dim)
print('単語数：', len(japanese_word2vec_vectors.itos))

100%|█████████▉| 1014939/1015474 [01:41<00:00, 10433.37it/s]

１単語を表現する次元数： 200
単語数： 1015474


100%|█████████▉| 1014939/1015474 [02:00<00:00, 10433.37it/s]

In [19]:
# ベクトル化したバージョンのボキャブラリーを作成
TEXT.build_vocab(train_ds, vectors=japanese_word2vec_vectors, min_freq=1)

# ボキャブラリーのベクトルを確認
print(TEXT.vocab.vectors.shape)
TEXT.vocab.vectors

# train_dsに含まれている単語49個がそれぞれ200次元のベクトルで表現されている

torch.Size([49, 200])


tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 2.6023, -2.6357, -2.5822,  ...,  0.6953, -1.4977,  1.4752],
        ...,
        [-2.8353,  2.5609, -0.5348,  ...,  0.4602,  1.4669, -2.1255],
        [-1.5885,  0.1614, -0.6029,  ..., -1.7545, -1.2462,  2.3034],
        [-0.0448, -0.1304,  0.0329,  ...,  0.0825, -0.1386,  0.0417]])

In [20]:
# ボキャブラリーの単語の順番を確認
TEXT.vocab.stoi

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7f4df580ec88>>,
            {'<unk>': 0,
             '<pad>': 1,
             'と': 2,
             '。': 3,
             'な': 4,
             'の': 5,
             '文章': 6,
             '、': 7,
             'が': 8,
             'し': 9,
             'を': 10,
             'いる': 11,
             'か': 12,
             'て': 13,
             'ます': 14,
             '分類': 15,
             '本章': 16,
             '評価': 17,
             '0': 18,
             'い': 19,
             'から': 20,
             'する': 21,
             'その': 22,
             'た': 23,
             'で': 24,
             'です': 25,
             'に': 26,
             'に対して': 27,
             'は': 28,
             'まし': 29,
             'クラス': 30,
             'ネガティブ': 31,
             'ポジティブ': 32,
             'モデル': 33,
             'レビュー': 34,
             '値': 35,
             '取り組み': 36,
             '商品': 37,
             '女性': 38,
     

In [25]:
# 単語のベクトル計算ができるか確認
import torch.nn.functional as F

# 姫 - 女性 + 男性
tensor_calc = TEXT.vocab.vectors[41] - \
    TEXT.vocab.vectors[38] + TEXT.vocab.vectors[46]

# コサイン類似度（ベクトルの距離を求める）
# dim=0は０次元目で計算してくださいとういう意味  dimの意味がわからない
print("女王", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[39], dim=0))
print("王", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[44], dim=0))
print("王子", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[45], dim=0))
print("機械学習", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[43], dim=0))


女王 tensor(0.3840)
王 tensor(0.3669)
王子 tensor(0.5489)
機械学習 tensor(-0.1404)


### fastText
word2vecより進歩している

In [26]:
from torchtext.vocab import Vectors

japanese_fasttext_vectors = Vectors(name='./data/vector_neologd/model.vec')

# 単語ベクトルの中身を確認
print('１単語を表現する次元数：', japanese_fasttext_vectors.dim)
print('単語数：', len(japanese_fasttext_vectors.itos))


  0%|          | 0/351122 [00:00<?, ?it/s]Skipping token b'351122' with 1-dimensional vector [b'300']; likely a header

 30%|███       | 107035/351122 [00:13<00:31, 7853.35it/s]


 60%|██████    | 211760/351122 [00:27<00:19, 7273.59it/s]


 91%|█████████ | 318563/351122 [00:41<00:03, 8150.21it/s]


100%|█████████▉| 350780/351122 [00:46<00:00, 6702.39it/s]

１単語を表現する次元数： 300
単語数： 351122



100%|█████████▉| 350780/351122 [00:59<00:00, 6702.39it/s]

In [27]:
# ベクトル化したバージョンのボキャブラリーを作成
TEXT.build_vocab(train_ds, vectors=japanese_fasttext_vectors, min_freq=1)

# ボキャブラリーのベクトルを確認
print(TEXT.vocab.vectors.shape)
TEXT.vocab.vectors

# train_dsに含まれている単語49個がそれぞれ300次元のベクトルで表現されている

torch.Size([49, 300])


tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0182, -0.0894, -0.1011,  ..., -0.0410, -0.0735,  0.0045],
        ...,
        [ 0.0161, -0.0172, -0.2945,  ..., -0.0038,  0.1588, -0.0076],
        [-0.2099,  0.1631, -0.0182,  ..., -0.4823,  0.0375,  0.0088],
        [-0.2762,  0.2986,  0.0511,  ..., -0.1052, -0.1779, -0.1957]])

In [28]:
# ボキャブラリーの単語の順番を確認
TEXT.vocab.stoi

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7f4db0933a90>>,
            {'<unk>': 0,
             '<pad>': 1,
             'と': 2,
             '。': 3,
             'な': 4,
             'の': 5,
             '文章': 6,
             '、': 7,
             'が': 8,
             'し': 9,
             'を': 10,
             'いる': 11,
             'か': 12,
             'て': 13,
             'ます': 14,
             '分類': 15,
             '本章': 16,
             '評価': 17,
             '0': 18,
             'い': 19,
             'から': 20,
             'する': 21,
             'その': 22,
             'た': 23,
             'で': 24,
             'です': 25,
             'に': 26,
             'に対して': 27,
             'は': 28,
             'まし': 29,
             'クラス': 30,
             'ネガティブ': 31,
             'ポジティブ': 32,
             'モデル': 33,
             'レビュー': 34,
             '値': 35,
             '取り組み': 36,
             '商品': 37,
             '女性': 38,
     

In [30]:
import torch.nn.functional as F

# 姫 - 女性 + 男性
tensor_calc = TEXT.vocab.vectors[41] - \
    TEXT.vocab.vectors[38] + TEXT.vocab.vectors[46]

# コサイン類似度（ベクトルの距離を求める）
# dim=0は０次元目で計算してくださいとういう意味  dimの意味がわからない
print("女王", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[39], dim=0))
print("王", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[44], dim=0))
print("王子", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[45], dim=0))
print("機械学習", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[43], dim=0))

女王 tensor(0.3650)
王 tensor(0.3461)
王子 tensor(0.5531)
機械学習 tensor(0.0952)
